# Flotorch ADK - Complete Workflow Example

This notebook demonstrates the complete workflow using FlotorchADKAgent with FlotorchMemoryService.
This combines everything from previous examples into one complete solution.

## What you'll learn
- Using FlotorchADKAgent (from example 4)
- Adding FlotorchMemoryService (from example 3)
- Complete end-to-end workflow

## Requirements
- FLOTORCH_API_KEY environment variable
- FLOTORCH_BASE_URL environment variable


In [46]:
# Setup and imports
import asyncio
import os
import sys
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

print("✓ Environment setup complete")


✓ Environment setup complete


In [48]:
# Import FlotorchADKAgent and FlotorchMemoryService
from flotorch.adk.agent import FlotorchADKAgent
from flotorch.adk.memory import FlotorchMemoryService
from google.adk.sessions import InMemorySessionService
from google.adk import Runner
from google.genai import types

print("✓ All imports successful")


✓ All imports successful


In [60]:
# Configuration
APP_NAME = "flotorch_agent"
USER_ID = "flotorch_user_123"
MEMORY_PROVIDER = "memo-provider"

print(f"App Name: {APP_NAME}")
print(f"User ID: {USER_ID}")
print(f"Memory Provider: {MEMORY_PROVIDER}")


App Name: flotorch_agent
User ID: flotorch_user_123
Memory Provider: memo-provider


## Step 1: Create Memory Service

First, we create the FlotorchMemoryService that will store our conversations.


In [61]:
# Create FlotorchMemoryService
memory_service = FlotorchMemoryService(
    api_key=os.getenv("FLOTORCH_API_KEY"),
    base_url=os.getenv("FLOTORCH_BASE_URL"),
    name=MEMORY_PROVIDER
)

print("✓ Memory service created")


✓ Memory service created


In [62]:
# Create FlotorchADKAgent with memory enabled
adk_client = FlotorchADKAgent(
    agent_name="flotorch_workflow_agent",
    enable_memory=True  # This is the key difference - memory is enabled
)

# Get the agent
agent = adk_client.get_agent()

print("✓ FlotorchADKAgent with memory created")


✓ FlotorchADKAgent with memory created


In [63]:
# Setup runner with both agent and memory service
session_service = InMemorySessionService()
runner = Runner(
    agent=agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service  # This connects memory to the runner
)

print("✓ Runner setup with memory service complete")


✓ Runner setup with memory service complete


In [64]:
# Helper function to send messages
def run_single_turn(query, session_id, user_id):
    """Send message to agent and get response."""
    content = types.Content(role="user", parts=[types.Part(text=query)])
    events = runner.run(user_id=user_id, session_id=session_id, new_message=content)

    for event in events:
        if event.is_final_response():
            if event.content and event.content.parts:
                return event.content.parts[0].text
    
    return "Sorry, I couldn't process that."

print("✓ Helper function ready")


✓ Helper function ready


## Test the Complete Workflow

Now let's test our complete setup with a conversation.


In [65]:
# Create session and start conversation
async def start_conversation():
    session = await runner.session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
    )
    
    print(f"Complete workflow ready! Session: {session.id}")
    return session

# Start the conversation
session = await start_conversation()


Complete workflow ready! Session: 5c89f86e-c07d-4adc-a013-822e6c22c880


In [66]:
# Simple chat function
def chat(message):
    """Chat with the complete workflow agent."""
    response = run_single_turn(message, session.id, USER_ID)
    print(f"You: {message}")
    print(f"Flotorch AI: {response}")
    

# Have a conversation
print("=== Testing Complete Workflow ===")
chat("Hello! I'm John and I work as a data scientist.")


=== Testing Complete Workflow ===
You: Hello! I'm John and I work as a data scientist.
Flotorch AI: Hello, John! It's great to meet you. As a data scientist, you must have some interesting projects. If there's anything specific you'd like to discuss or any questions you have, feel free to share!


In [67]:
# Continue the conversation
print("\n" + "-"*40)
chat("I'm working on a machine learning project about customer behavior.")



----------------------------------------
You: I'm working on a machine learning project about customer behavior.
Flotorch AI: That sounds fascinating, John! Understanding customer behavior can provide valuable insights for businesses. Are you focusing on any particular aspects of customer behavior, like purchasing patterns or engagement metrics?


In [68]:
# Store the conversation in memory
async def save_conversation():
    print("Saving conversation to memory...")
    
    completed_session = await runner.session_service.get_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=session.id
    )
    
    await memory_service.add_session_to_memory(completed_session)
    print("✓ Conversation saved to memory!")

# Save the conversation
await save_conversation()


Saving conversation to memory...
✓ Conversation saved to memory!


## Test Memory Recall

Let's create a new session to see if the agent remembers our previous conversation.


In [69]:
# Create new session to test memory
async def test_memory():
    new_session = await runner.session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
    )
    
    print(f"New session created: {new_session.id}")
    
    # Test if agent remembers
    response = run_single_turn("What do you remember about me?", new_session.id, USER_ID)
    print(f"You: What do you remember about me?")
    print(f"Flotorch AI: {response}")

# Test memory recall
await test_memory()


New session created: 7c2f2bdc-b961-431d-9ce6-2ef5c28568e4
You: What do you remember about me?
Flotorch AI: I remember that your name is John and you work as a data scientist. If there's anything specific you'd like to discuss or ask about, feel free to let me know!


## Summary

This complete workflow demonstrates:
- ✓ FlotorchADKAgent with memory enabled
- ✓ FlotorchMemoryService for storing conversations
- ✓ Memory recall across different sessions
- ✓ Complete integration of all components

**This combines everything from examples 1-4 into one complete solution!**

You can use `chat("your message")` to continue chatting with the complete workflow.
